# FastText Model
## Treinamento do modelo
### Imports

In [ ]:

import pandas as pd
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
import re
from unidecode import unidecode
import time
import json
import fasttext
from tqdm import tqdm
from collections import defaultdict

In [17]:
df_clean = pd.read_parquet('train_clean.parquet')
with open('variacoes_palavras.json', 'r', encoding='utf-8') as f:
    variacoes_palavras = json.load(f)

### Criação da base de conhecimento
utilizada as colunas para aprimorar o vocabulário base do fasttext.\
composto das colunas do df + variaçoes criadas por erros de escrita no pre processamento dos dados

In [18]:
with open('fasttext_train.txt', 'w', encoding='utf-8') as f:
    for text in df_clean['razaosocial_clean'].dropna():
        f.write(text + '\n')
        
    for text in df_clean['nome_fantasia_clean'].dropna():
        f.write(text + '\n')
    
    for text in df_clean['user_input_clean'].dropna():
        f.write(text + '\n')
        
     # UFs (coluna permitida)
    for uf in df_clean['uf'].dropna().unique():
        f.write(str(uf) + '\n')   
        
    for main_word, variations in variacoes_palavras.items():
        for variation in variations:
            f.write(variation + '\n')

função de limpa e padronização um arquivo texto para que ele:

1. Não tenha linhas vazias.
2. Tenha apenas caracteres ASCII (sem acentos, símbolos ou emojis).
3. Tenha uma linha por entrada com texto limpo.

In [19]:
def clean_file(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as f_in:
        with open(output_path, 'w', encoding='utf-8') as f_out:
            for i, line in enumerate(f_in):
                # Verificar linhas problemáticas
                if not line.strip():
                    continue  # Ignorar linhas vazias
                
                if any(ord(char) > 127 for char in line):
                    # Corrigir caracteres especiais
                    line = ''.join(char for char in line if ord(char) < 128)
                
                f_out.write(line.strip() + '\n')

clean_file('fasttext_train.txt', 'fasttext_train_clean.txt')

Foram testados múltiplas configurações para o modelo

In [ ]:
model = fasttext.train_unsupervised(
    input='fasttext_train.txt',
    model='skipgram',
    dim=300,            
    epoch=75,           
    minCount=2,         
    minn=3,             
    maxn=5,             
    wordNgrams=3,
    lr=0.1,            # Taxa de aprendizado
    thread=32,         # Utiliza todos os 8 cores 4 threads por core
    bucket=5000000,     
    loss='hs',         
    verbose=2       

)


### Salvamento do modelo treinado

In [21]:
model.save_model('modelo.bin')

## Avaliação do
### Teste da acurácia do modelo
Encontrar palavras utilizando a tecnica de nearest neighbors

In [ ]:


def evaluate_model(model, test_cases, timeout_minutes=30):
    acertos = 0
    total_testes = 0
    start_time = time.time()
    
    # Criar lista plana de testes
    test_list = []
    for palavra, variacoes in test_cases.items():
        for variacao in variacoes:
            test_list.append((palavra, variacao))
    
    # Avaliar com barra de progresso
    for palavra, variacao in tqdm(test_list, desc="Avaliando modelo"):
        try:
            similares = model.get_nearest_neighbors(variacao, k=5)
            palavras_similares = [p for _, p in similares]
            
            if palavra in palavras_similares:
                acertos += 1
                
            total_testes += 1
            
            # Verificar timeout
            elapsed = (time.time() - start_time) / 60
            if elapsed > timeout_minutes:
                print(f"\nTimeout atingido após {timeout_minutes} minutos")
                break
                
        except Exception as e:
            print(f"\nErro ao processar '{variacao}': {str(e)}")
            continue
    
    # Retornar ambos os valores
    acuracia = acertos / total_testes if total_testes > 0 else 0.0
    return acuracia, total_testes

acuracia, total_testes = evaluate_model(model, variacoes_palavras, timeout_minutes=15)
print(f"Acurácia: {acuracia:.2%} | Testes realizados: {total_testes}")

Avaliando modelo:  24%|██▍       | 164657/690695 [14:59<47:54, 182.98it/s]  


Timeout atingido após 15 minutos
Acurácia: 77.11% | Testes realizados: 164658


#### Criação de chave única por empresa
composta de razao_social + nome fantasia

In [ ]:
# Criação do dicionário de empresas únicas
empresas_embeddings = []
chaves_unicas = set()

for _, row in df_clean.iterrows():
    razao = row['razaosocial_clean'] or ""
    fantasia = row['nome_fantasia_clean'] or ""
    chave = f"{razao}|{fantasia}"
    
    # Se já processamos essa combinação, pular
    if chave in chaves_unicas:
        continue
        
    chaves_unicas.add(chave)
    
    texto_empresa = f"{razao} {fantasia}".strip()
    palavras = texto_empresa.split()
    vetores = []
    
    for p in palavras:
        if p in model.words:
            vetores.append(model.get_word_vector(p))
    
    # Calcular embedding médio
    if vetores:
        embedding_medio = np.mean(vetores, axis=0)
    else:
        embedding_medio = np.zeros(model.get_dimension())
    
    empresas_embeddings.append({
        'razao_social': razao,
        'nome_fantasia': fantasia,
        'embedding': embedding_medio
    })

print(f"Total de empresas únicas: {len(empresas_embeddings)}")

Total de empresas únicas: 9905


#### Buscar empresas mais similares

In [24]:

def buscar_empresas(query, top_n=5):
    # Pré-processar a query
    palavras = query.lower().split()
    vetores = []
    
    # Obter vetores para palavras válidas
    for p in palavras:
        if p in model.words:
            vetores.append(model.get_word_vector(p))
    
    if not vetores:
        return []
    
    # Calcular embedding médio da query
    query_embed = np.mean(vetores, axis=0)
    
    # Normalização CRUCIAL para similaridade correta
    norm_query = np.linalg.norm(query_embed)
    if norm_query == 0:
        return []
    query_embed_norm = query_embed / norm_query
    
    resultados = []
    empresas_vistas = set()
    
    # Calcular similaridade para cada empresa
    for empresa in empresas_embeddings:
        # Criar chave única para evitar duplicatas
        chave = f"{empresa['razao_social']}|{empresa['nome_fantasia']}"
        if chave in empresas_vistas:
            continue
        empresas_vistas.add(chave)
        
        emb = empresa['embedding']
        norm_emb = np.linalg.norm(emb)
        
        # Ignorar empresas com embedding zerado
        if norm_emb == 0:
            continue
            
        # Calcular similaridade de cosseno
        emb_norm = emb / norm_emb
        sim = np.dot(emb_norm, query_embed_norm)
        
        resultados.append({
            'razao_social': empresa['razao_social'],
            'nome_fantasia': empresa['nome_fantasia'],
            'similaridade': sim  # Agora entre -1 e 1
        })
    
    # Ordenar resultados por similaridade decrescente
    resultados_ordenados = sorted(resultados, key=lambda x: x['similaridade'], reverse=True)
    return resultados_ordenados[:top_n]

In [25]:
def testar_busca(query, top_n=5):
    print(f"\n--- Testando: '{query}' ---")
    palavras_validas = [p for p in query.split() if p in model.words]
    print(f"Palavras válidas no modelo: {palavras_validas}")
    
    resultados = buscar_empresas(query, top_n)
    
    if not resultados:
        print("Nenhum resultado encontrado")
        return
        
    for i, res in enumerate(resultados, 1):
        print(f"{i}. {res['nome_fantasia']} | Similaridade: {res['similaridade']:.4f}")
        
        # Mostrar razão social apenas se for diferente do nome fantasia
        if res['razao_social'] and res['razao_social'] != res['nome_fantasia']:
            print(f"   Razão Social: {res['razao_social']}")
    
    return resultados

##### Teste da capacidade de busca do modelo

In [26]:
testes = ["magazine luiz", "posto guerra", "farmaceutica"]
for query in testes:
    testar_busca(query)


--- Testando: 'magazine luiz' ---
Palavras válidas no modelo: ['magazine', 'luiz']
1. magazine luzia | Similaridade: 0.7686
   Razão Social: magazine luiza
2. magazine luiza | Similaridade: 0.7506
3. magazineluiza | Similaridade: 0.7438
   Razão Social: magazine luiza
4. imperio magazine | Similaridade: 0.6032
5. sol magazine | Similaridade: 0.6004
   Razão Social: sol magazine uruacu

--- Testando: 'posto guerra' ---
Palavras válidas no modelo: ['posto', 'guerra']
1. posto luiz guerra | Similaridade: 0.8833
   Razão Social: auto posto luiz guerra
2. galvao guerra | Similaridade: 0.7181
3. auto posto wr | Similaridade: 0.6283
4. auto posto marciano | Similaridade: 0.6278
5. auto posto formigao | Similaridade: 0.6100

--- Testando: 'farmaceutica' ---
Palavras válidas no modelo: ['farmaceutica']
1. inovat | Similaridade: 0.6121
   Razão Social: uniao quimica farmaceutica
2. drogal brodowski | Similaridade: 0.5853
   Razão Social: drogal farmaceutica
3. rd farma | Similaridade: 0.5752
  

### Classificação do Dataframe

In [ ]:
# Pré-calcular embeddings para todas as empresas únicas
print("Pré-calculando embeddings para empresas únicas...")
empresas_unicas = df_clean[['razaosocial_clean', 'nome_fantasia_clean']].drop_duplicates()

# Dicionário para embeddings das empresas
emp_embeddings = {}
for _, row in tqdm(empresas_unicas.iterrows(), total=len(empresas_unicas)):
    razao = row['razaosocial_clean'] or ""
    fantasia = row['nome_fantasia_clean'] or ""
    texto = f"{razao} {fantasia}".strip()
    palavras = texto.split()
    vetores = [model.get_word_vector(p) for p in palavras if p in model.words]
    
    if vetores:
        embedding = np.mean(vetores, axis=0)
    else:
        embedding = np.zeros(model.get_dimension())
    
    emp_embeddings[(razao, fantasia)] = embedding

# Pré-calcular embeddings para todas as queries únicas
print("Pré-calculando embeddings para queries únicas...")
queries_unicas = df_clean['user_input_clean'].dropna().unique()
query_embeddings = {}
for query in tqdm(queries_unicas):
    palavras = query.split()
    vetores = [model.get_word_vector(p) for p in palavras if p in model.words]
    
    if vetores:
        embedding = np.mean(vetores, axis=0)
    else:
        embedding = np.zeros(model.get_dimension())
    
    query_embeddings[query] = embedding

# Criar matriz de embeddings para cálculo eficiente
print("Preparando matrizes para cálculo vetorizado...")
# Converter para arrays numpy
all_emp_keys = list(emp_embeddings.keys())
emp_matrix = np.array([emp_embeddings[k] for k in all_emp_keys])

all_query_keys = list(query_embeddings.keys())
query_matrix = np.array([query_embeddings[k] for k in all_query_keys])

# Normalizar embeddings para similaridade de cosseno eficiente
emp_norms = np.linalg.norm(emp_matrix, axis=1, keepdims=True)
emp_norms[emp_norms == 0] = 1e-9  # Evitar divisão por zero
emp_matrix_norm = emp_matrix / emp_norms

query_norms = np.linalg.norm(query_matrix, axis=1, keepdims=True)
query_norms[query_norms == 0] = 1e-9
query_matrix_norm = query_matrix / query_norms

# Calcular matriz de similaridade em blocos (para economizar memória)
def calcular_similaridades_em_blocos(query_matrix, emp_matrix, block_size=1000):
    """Calcula similaridades em blocos para evitar estouro de memória"""
    num_queries = query_matrix.shape[0]
    similarity_matrix = np.zeros((num_queries, emp_matrix.shape[0]))
    
    for start in tqdm(range(0, num_queries, block_size)):
        end = min(start + block_size, num_queries)
        block = query_matrix[start:end]
        sim_block = np.dot(block, emp_matrix.T)
        similarity_matrix[start:end] = sim_block
    
    return similarity_matrix

print("Calculando matriz de similaridade completa...")
similarity_matrix = calcular_similaridades_em_blocos(query_matrix_norm, emp_matrix_norm)

# Mapear índices para busca rápida
query_idx_map = {query: idx for idx, query in enumerate(all_query_keys)}
emp_idx_map = {emp: idx for idx, emp in enumerate(all_emp_keys)}

# Função para avaliar todo o DataFrame
def avaliar_dataframe_completo(df):
    top1_correct = 0
    top5_correct = 0
    total = 0
    
    for _, row in tqdm(df.iterrows(), total=len(df)):
        query = row['user_input_clean']
        razao_esperada = row['razaosocial_clean']
        fantasia_esperada = row['nome_fantasia_clean']
        emp_esperada = (razao_esperada, fantasia_esperada)
        
        # Pular linhas sem query válida
        if pd.isna(query) or not query:
            continue
        
        # Obter índices
        query_idx = query_idx_map.get(query)
        emp_idx = emp_idx_map.get(emp_esperada)
        
        # Se não encontrou algum dos embeddings, pular
        if query_idx is None or emp_idx is None:
            continue
        
        # Obter similaridades para esta query
        query_similarities = similarity_matrix[query_idx]
        
        # Encontrar top 5 empresas para esta query
        top5_indices = np.argsort(query_similarities)[::-1][:5]
        
        # Verificar se a empresa esperada está no top 1 e top 5
        if top5_indices[0] == emp_idx:
            top1_correct += 1
            top5_correct += 1
        elif emp_idx in top5_indices:
            top5_correct += 1
        
        total += 1
    
    return top1_correct, top5_correct, total

# Executar avaliação completa
print("\nIniciando avaliação completa do DataFrame...")
top1_correct, top5_correct, total_valid = avaliar_dataframe_completo(df_clean)

# Calcular métricas finais
top1_accuracy = top1_correct / total_valid if total_valid > 0 else 0
top5_accuracy = top5_correct / total_valid if total_valid > 0 else 0

# Resultados finais
print("\n" + "="*60)
print(f"Resultados da Avaliação Completa (Total de registros válidos: {total_valid})")
print(f"Top-1 Accuracy: {top1_accuracy:.4f} ({top1_correct}/{total_valid})")
print(f"Top-5 Accuracy: {top5_accuracy:.4f} ({top5_correct}/{total_valid})")
print("="*60)

Pré-calculando embeddings para empresas únicas...


100%|██████████| 9905/9905 [00:02<00:00, 4010.12it/s]


Pré-calculando embeddings para queries únicas...


100%|██████████| 78702/78702 [00:08<00:00, 9402.72it/s] 


Preparando matrizes para cálculo vetorizado...
Calculando matriz de similaridade completa...


100%|██████████| 79/79 [00:10<00:00,  7.32it/s]



Iniciando avaliação completa do DataFrame...


100%|██████████| 255471/255471 [02:49<00:00, 1505.91it/s]


Resultados da Avaliação Completa (Total de registros válidos: 255392)
Top-1 Accuracy: 0.4931 (125934/255392)
Top-5 Accuracy: 0.7535 (192441/255392)
Resultados salvos em 'resultados_avaliacao.txt'
